In [83]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statistics
from statsmodels.tsa.seasonal import STL


In [84]:
consumer1 = pd.read_csv('./data/Fluvius load profiles type 1 consumer/Type1consumerLoad861.csv', header=None)
# kWh -> kW
consumer1 = consumer1*4
# Rename columns to start numbering at 1
consumer1.columns = range(1, consumer1.shape[1] + 1)
# Adjust the index to start at 1
consumer1.index = range(1, len(consumer1) + 1)

consumer2 = pd.read_csv('./data/Fluvius load profiles type 1 consumer/Type1consumerLoad464.csv', header=None)
consumer2 = consumer2*4
consumer2.columns = range(1, consumer2.shape[1] + 1)
consumer2.index = range(1, len(consumer2) + 1)

consumer3 = pd.read_csv('./data/Fluvius load profiles type 1 consumer/Type1consumerLoad540.csv', header=None)
consumer3 = consumer3*4
consumer3.columns = range(1, consumer3.shape[1] + 1)
consumer3.index = range(1, len(consumer3) + 1)

consumer4 = pd.read_csv('./data/Fluvius load profiles type 1 consumer/Type1consumerLoad1029.csv', header=None)
consumer4 = consumer4*4
consumer4.columns = range(1, consumer4.shape[1] + 1)
consumer4.index = range(1, len(consumer4) + 1)

def min_max_normalize(df):
    return (df - df.min()) / (df.max() - df.min())
chargingprofile1 = pd.read_csv('./data/yearly_charging_profile_1.csv', header=None)
# Drop the first row and column (headers)
chargingprofile1 = chargingprofile1.drop(index=0, columns=0).reset_index(drop=True)
# Adjust the index to start at 1
chargingprofile1.index = range(1, len(chargingprofile1) + 1)
# Apply normalization
chargingprofile1 = min_max_normalize(chargingprofile1)

chargingprofile2 = pd.read_csv('./data/yearly_charging_profile_2.csv', header=None)
chargingprofile2 = chargingprofile2.drop(index=0, columns=0).reset_index(drop=True)
chargingprofile2.index = range(1, len(chargingprofile2) + 1)
chargingprofile2 = min_max_normalize(chargingprofile2)

chargingprofile3 = pd.read_csv('./data/yearly_charging_profile_3.csv', header=None)
chargingprofile3 = chargingprofile3.drop(index=0, columns=0).reset_index(drop=True)
chargingprofile3.index = range(1, len(chargingprofile3) + 1)
chargingprofile3 = min_max_normalize(chargingprofile3)

solargen = pd.read_excel('./data/Belgium_solar_generation_probability_density.xlsx', sheet_name='Solar_profiles',header=None)
# kWh -> kW
solargen = solargen
# Drop the first row and column (headers)
solargen = solargen.drop(index=0, columns=0).reset_index(drop=True)
# Adjust the index to start at 1
solargen.index = range(1, len(solargen) + 1)
# Drop the last column: 366 days -> 365 days
solargen = solargen.iloc[:, :-1]


Min-median

In [85]:
# Average daily load profiles
consumer_profiles = [consumer1, consumer2, consumer3]
results_mean_ev = {0: [], 1: [], 2: []}
results_std_ev = {0: [], 1: [], 2: []}

results_mean_pv = {0: [], 1: [], 2: []}
results_std_pv = {0: [], 1: [], 2: []}


kw_values = np.arange(1, 24, 1)  # From 1 to 23 kW, incrementing by 1
kwp_values = np.arange(1, 11, 1) 

for i, consumer_profile in enumerate(consumer_profiles):
    for kw in kw_values: 
        net_consumption = consumer_profile + (kw * chargingprofile1)
        net_consumption_mean = net_consumption.median(axis=1)
        #min_power_ev = net_consumption.min(axis=1)
        net_consumption_std = net_consumption.std(axis=1)

        results_mean_ev[i].append(net_consumption_mean)
        results_std_ev[i].append(net_consumption_std)

    for kwp in kwp_values:
        net_consumption = consumer_profile - (kwp * solargen)
        #net_consumption_mean = net_consumption.mean(axis=1)
        net_consumption_mean = net_consumption.median(axis=1)
        net_consumption_std = net_consumption.std(axis=1)

        results_mean_pv[i].append(net_consumption_mean)
        results_std_pv[i].append(net_consumption_std)



In [86]:
# Min of average daily load profiles
print('Minimum of average daily load profiles')
print('Baseload:')
print(consumer1.median(axis=1).min(),consumer2.median(axis=1).min(), consumer3.median(axis=1).min())
print('\n')
print('Baseload - pv:')
for consumer in results_mean_pv:
    print('Min mean of consumer',consumer+1,' per kWp: ')
    rounded_values = [round(results_mean_pv[consumer][kw].min(), 2) for kw in range(len(results_mean_pv[consumer]))]
    print(*rounded_values)
print('\n')
print('Baseload + ev:')
for consumer in results_mean_ev:
    print('Min mean of consumer',consumer+1,' per charger kW: ')
    rounded_values = [round(results_mean_ev[consumer][kw].min(), 2) for kw in range(len(results_mean_ev[consumer]))]
    print(*rounded_values)
print('\n')

Minimum of average daily load profiles
Baseload:
0.092 0.164 0.284


Baseload - pv:
Min mean of consumer 1  per kWp: 
-0.37 -0.9 -1.42 -1.98 -2.52 -3.07 -3.63 -4.2 -4.77 -5.31
Min mean of consumer 2  per kWp: 
-0.28 -0.83 -1.37 -1.94 -2.47 -3.01 -3.56 -4.1 -4.65 -5.2
Min mean of consumer 3  per kWp: 
0.28 -0.11 -0.65 -1.17 -1.68 -2.23 -2.84 -3.37 -3.88 -4.42


Baseload + ev:
Min mean of consumer 1  per charger kW: 
0.09 0.09 0.09 0.09 0.09 0.09 0.09 0.09 0.09 0.09 0.09 0.09 0.09 0.09 0.09 0.09 0.09 0.09 0.09 0.09 0.09 0.09 0.09
Min mean of consumer 2  per charger kW: 
0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16
Min mean of consumer 3  per charger kW: 
0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29 0.29




In [87]:
# Max std of min daily load profiles
print('Max std of power across different days')
print('Baseload:')
print(consumer1.std(axis=1).max(),consumer2.std(axis=1).max(), consumer3.std(axis=1).max())
print('\n')

print('Baseload - pv:')
for consumer in results_std_pv:
    print('Consumer',consumer+1,' per kWp: ')
    rounded_values = [round(results_std_pv[consumer][kw].max(), 2) for kw in range(len(results_std_pv[consumer]))]
    print(*rounded_values)
print('\n')
print('Baseload + ev:')
for consumer in results_std_ev:
    print('Consumer',consumer+1,' per charger kW: ')
    rounded_values = [round(results_std_ev[consumer][kw].max(), 2) for kw in range(len(results_std_ev[consumer]))]
    print(*rounded_values)
print('\n')

Max std of power across different days
Baseload:
0.4222095311014933 0.667545031973484 1.0252822156543857


Baseload - pv:
Consumer 1  per kWp: 
0.48 0.69 0.96 1.25 1.54 1.84 2.14 2.44 2.74 3.04
Consumer 2  per kWp: 
0.76 0.93 1.14 1.37 1.67 1.97 2.27 2.57 2.87 3.18
Consumer 3  per kWp: 
0.97 1.0 1.1 1.32 1.59 1.86 2.15 2.43 2.72 3.01


Baseload + ev:
Consumer 1  per charger kW: 
0.57 0.86 1.19 1.53 1.89 2.25 2.62 2.99 3.36 3.73 4.1 4.47 4.84 5.21 5.58 5.95 6.32 6.69 7.06 7.43 7.8 8.17 8.54
Consumer 2  per charger kW: 
0.76 1.0 1.29 1.62 1.96 2.3 2.66 3.01 3.37 3.74 4.1 4.47 4.84 5.21 5.58 5.95 6.32 6.69 7.06 7.43 7.8 8.17 8.54
Consumer 3  per charger kW: 
1.06 1.21 1.44 1.72 2.02 2.35 2.7 3.06 3.43 3.79 4.16 4.53 4.89 5.26 5.63 6.0 6.37 6.74 7.11 7.48 7.85 8.21 8.58




In [88]:
print('First autocorrelation coefficient of remainder series in STL')
print('Baseload')
for consumer in consumer_profiles:
    series = consumer.values.flatten()
    time_series = pd.Series(series)
    # Apply STL decomposition (seasonal period = 96 for daily patterns)
    stl = STL(time_series, period=96, robust=True)
    result = stl.fit()
    remainder_series = result.resid
    first_autocorr_coeff = remainder_series.autocorr(lag=1)
    print(first_autocorr_coeff)
print('\n')

print('Baseload - pv:')
kw_values = [1, 11, 23]  # From 1 to 23 kW, incrementing by 1
kwp_values = [1, 5, 10]  # From 1 to 23 kW, incrementing by 1
for i, consumer_profile in enumerate(consumer_profiles):
    for kwp in kwp_values:
        net_consumption = consumer_profile - (kwp * solargen)
        series = net_consumption.values.flatten()
        time_series = pd.Series(series)
        # Apply STL decomposition (seasonal period = 96 for daily patterns)
        stl = STL(time_series, period=96, robust=True)
        result = stl.fit()
        remainder_series = result.resid
        first_autocorr_coeff = remainder_series.autocorr(lag=1)
        print(first_autocorr_coeff)
    print('\n')

    print('Baseload + ev:')
    for kw in kw_values: 
        net_consumption = consumer_profile + (kw * chargingprofile1)
        series = net_consumption.values.flatten()
        time_series = pd.Series(series)
        # Apply STL decomposition (seasonal period = 96 for daily patterns)
        stl = STL(time_series, period=96, robust=True)
        result = stl.fit()
        remainder_series = result.resid
        first_autocorr_coeff = remainder_series.autocorr(lag=1)
        print(first_autocorr_coeff)



First autocorrelation coefficient of remainder series in STL
Baseload
0.04738285116191077
0.28777084314296253
0.15706840382479045


Baseload - pv:
0.13737295087549428
0.5095523779192596
0.5436459002675121


Baseload + ev:
0.04499205030469394
0.04686024700081933
0.04315096208055813
0.3216427299889743
0.5248329755254448
0.5414645402160682


Baseload + ev:
0.18937024255015597
0.05378990425452019
0.04830197618107317
0.18635237611002822
0.3746293285794352
0.46570680481371374


Baseload + ev:
0.1519927499971096
0.05027927229350559
0.046001552161794856
